In [ ]:
import pyomo.environ as pyo
from solar_installations import INSTALLATION_SIZES

In [ ]:
model = pyo.ConcreteModel()
model.SOLAR_SIZES = pyo.Set(initialize=INSTALLATION_SIZES.keys())
model.solar_size_flags = pyo.Var(model.SOLAR_SIZES, within=pyo.Binary)
model.solar_capacity = pyo.Var(within=pyo.NonNegativeReals)
model.solar_cost = pyo.Var(within=pyo.NonNegativeReals)


def installed_solar_constraint(model):
    sum_term = sum(
        INSTALLATION_SIZES[i] * model.solar_size_flags[i] for i in model.SOLAR_SIZES
    )
    return model.solar_cost == sum_term


model.consumption_constraint = pyo.Constraint(rule=installed_solar_constraint)
model.sos1_constraint = pyo.SOSConstraint(var=model.solar_size_flags, sos=1)


def required_power(model):
    return model.solar_capacity >= 6


model.required_power_constraint = pyo.Constraint(rule=required_power)


def solar_capacity_constraint(model):
    return model.solar_capacity == sum(
        size * model.solar_size_flags[size] for size in model.SOLAR_SIZES
    )


model.solar_capacity_constraint = pyo.Constraint(rule=solar_capacity_constraint)
model.objective = pyo.Objective(expr=model.solar_cost, sense=pyo.maximize)
optimizer = pyo.SolverFactory("gurobi")
optimizer.solve(model)
print(model.display())